# 建构`RAG`应用

在之前的[RAG快速入门](../doc/RAG快速入门.md)中，我们已经了解到一个`RAG`应用包括如下部分：

- 索引
  - 文档加载，从不同数据源（或数据格式）加载数据
  - 文本拆分，将大块数据分割为更小的块，对每一个小块计算向量值
  - 存储，使用向量数据块来存储和索引向量值，为后续提供搜索
- 检索和生成
  - 给定的用户输入，从存储中检索相关分割的数据
  - 使用用户的问题和检索的数据为提示词生成答案

下面使用`langchain`来实现`RAG`。

## 文档加载

文档加载器实现了`BaseLoader`接口，使用`load`方法或调用`lazy_load`（大型数据集）。

主要的[文档加载器](https://python.langchain.com/docs/integrations/document_loaders/)：

- `PyPDF`: 加载和解析 `PDF`
- `CSVLoader`: 加载和解析 `csv`
- `JSONLoader`: 加载和解析 `json`
- `BSHTMLLoader`: 加载和解析 `HTML`
- `Docling`: 加载和解析 `PDF`, `DOCX`, `PPTX`, `HTML`等
- 各种网络在线文档等

下面主要讲述`PDF`文件的加载。

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('')
pages = []
for page in loader.lazy_load():
    pages.append(page)

## 文本分割

拆分文档主要是克服模型最大输入数的限制，同时可以提高检索精度，拆分的策略包括如下几种。

### 基于长度分割

包含两种方式：

- Character-based：以字符（如字母、标点、空格等） 为单位计算长度，根据字符数量来拆分文本。如：按\n\n（空行）作为分隔符，当累计字符数达到设定阈值时，将文本拆分为一段
  - 适合：通用文本拆分、非英文文本、需要保持原始字符结构的场景（如代码片段）
- Token-based：以Token为单位计算长度，根据 Token 数量来拆分文本。如：使用 OpenAI 的 Tokenizer 时，"Hello, world!" 会被拆分为 ["Hello", ",", "world", "!"]，共 4 个 Token。
  - 适合：需要严格遵守大模型 Token 限制的场景